In [1]:
import re
import pandas as pd

In [2]:
pattern = r"#### ((-?\$\s*[0-9.,]+)|(-?\$?[0-9.,]+))"


def extract_number(text):
    match = re.match(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

test_cases = [
    ("#### 123", "123"),
    ("#### 1.23", "1.23"),
    ("#### $1.23", "$1.23"),
    ("#### $ 1.23", "$ 1.23"),
    ("#### 12,000", "12,000"),
    ("#### 12000", "12000"),
]

for text, expected in test_cases:
    assert extract_number(text) == expected, f"Expected {expected}, got {extract_number(text)}"


In [3]:
df = pd.read_json('flat-results.jsonl', lines=True)
df

,doc_id,question,answer,target,model_response,filtered_response,exact_match
0,0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18,"<reasoning>\nFirst, we need to determine the t...",$20.,0
1,1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3,<reasoning>\nTo determine the total number of ...,[invalid],0
2,2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000,"<reasoning>\nTo determine Josh's profit, we ne...",[invalid],0
3,3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540,<reasoning>\nJames runs 3 sprints each time he...,$20.,0
4,4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20,<reasoning>\nWendi feeds each chicken 3 cups o...,[invalid],0
...,...,...,...,...,...,...,...
59,59,Cynthia eats one serving of ice cream every ni...,Each container of ice cream has 15 servings an...,16,<reasoning>\nCynthia eats one serving of ice c...,16,1
60,60,Henry made two stops during his 60-mile bike t...,He traveled 20 miles + 15 miles = <<20+15=35>>...,25,<reasoning>\nTo determine how many miles Henry...,$20.,0
61,61,Gloria is shoe shopping when she comes across ...,The second pair of heels costs 33 * 2 = $<<33*...,104,<reasoning>\nLet's denote the cost of the boot...,104,1
62,62,Darrell and Allen's ages are in the ratio of 7...,The total ratio representing their ages is 7+1...,109,<reasoning>\nLet's denote Darrell's age as \(7...,109,1


In [20]:
pattern = r"<answer>\s*([-$0-9.,]+)\s*</answer>"

def extract_answer(text):
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[0].strip()
    return None


In [21]:
df['new_filtered_response'] = df['model_response'].apply(extract_answer)
df

,doc_id,question,answer,target,model_response,filtered_response,exact_match,new_filtered_response,EM
0,0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...,18,"<reasoning>\nFirst, we need to determine the t...",$20.,0,18,False
1,1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...,3,<reasoning>\nTo determine the total number of ...,[invalid],0,3,False
2,2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...,70000,"<reasoning>\nTo determine Josh's profit, we ne...",[invalid],0,295000,False
3,3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...,540,<reasoning>\nJames runs 3 sprints each time he...,$20.,0,540,False
4,4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t...",20,<reasoning>\nWendi feeds each chicken 3 cups o...,[invalid],0,20,False
...,...,...,...,...,...,...,...,...,...
59,59,Cynthia eats one serving of ice cream every ni...,Each container of ice cream has 15 servings an...,16,<reasoning>\nCynthia eats one serving of ice c...,16,1,16,False
60,60,Henry made two stops during his 60-mile bike t...,He traveled 20 miles + 15 miles = <<20+15=35>>...,25,<reasoning>\nTo determine how many miles Henry...,$20.,0,25,False
61,61,Gloria is shoe shopping when she comes across ...,The second pair of heels costs 33 * 2 = $<<33*...,104,<reasoning>\nLet's denote the cost of the boot...,104,1,104,False
62,62,Darrell and Allen's ages are in the ratio of 7...,The total ratio representing their ages is 7+1...,109,<reasoning>\nLet's denote Darrell's age as \(7...,109,1,109,False


In [25]:
df['EM'] = df.apply(lambda row: row['new_filtered_response'] == str(row['target']), axis=1)
df['EM'].mean(), df['exact_match'].mean()

(np.float64(0.75), np.float64(0.234375))